In [9]:
import requests
import json
import time
import csv
from datetime import datetime

annict_token = ""
start_date = datetime.fromisoformat("2024-01-01T00:00:00")
end_date = datetime.fromisoformat("2024-12-31T23:59:59")
filtered_data = []
count = 0
flag = 0

for h in range(10000):
    url = f"https://api.annict.com/v1/activities?access_token={annict_token}&sort_id=desc&filter_username=ssa_yuuharu&fields=action,created_at,work.title,status.kind&per_page=50&page={h+1}"
    
    response = requests.get(url)
    
    # ステータスコードチェック
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        print("Response:", response.text)
        break

    # JSONデコード前の空チェック
    if response.text.strip() == "":
        print("Empty response from API")
        break

    try:
        data = json.loads(response.text)
    except json.JSONDecodeError as e:
        print("Failed to decode JSON:", e)
        print("Response text:", response.text)
        break

    # レスポンスに "activities" が含まれていない場合
    if "activities" not in data or data["activities"] == []:
        break

    for i in data["activities"]:
        if i["action"] == "create_status" and i["status"]["kind"] == "watched":
            created_at_object = datetime.fromisoformat(i["created_at"].replace("Z", "")).replace(tzinfo=None)
            
            if start_date <= created_at_object <= end_date:
                count += 1
                filtered_data.append(i)
            elif created_at_object < start_date:
                flag = 1
                break

    if flag == 1:
        break
    
    time.sleep(1)

# CSVファイルに書き込む
csv_file = "filtered_data.csv"
csv_columns = ["created_at", "work_title"]
try:
    with open(csv_file, 'w', newline='', encoding='utf-8-sig') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in filtered_data:
            writer.writerow({
                "created_at": data["created_at"],
                "work_title": data["work"]["title"]
            })
except IOError:
    print("I/O error")

print(filtered_data)
print(f"Total count: {count}")


[{'user': {}, 'action': 'create_status', 'created_at': '2024-12-31T10:26:39Z', 'work': {'title': 'ストライク・ザ・ブラッド 消えた聖槍篇'}, 'status': {'kind': 'watched'}}, {'user': {}, 'action': 'create_status', 'created_at': '2024-12-31T10:26:32Z', 'work': {'title': 'ストライク・ザ・ブラッド ヴァルキュリアの王国篇'}, 'status': {'kind': 'watched'}}, {'user': {}, 'action': 'create_status', 'created_at': '2024-12-31T09:34:46Z', 'work': {'title': 'ストライク・ザ・ブラッドⅢ'}, 'status': {'kind': 'watched'}}, {'user': {}, 'action': 'create_status', 'created_at': '2024-12-31T07:53:53Z', 'work': {'title': 'ストライク・ザ・ブラッドII'}, 'status': {'kind': 'watched'}}, {'user': {}, 'action': 'create_status', 'created_at': '2024-12-28T03:29:32Z', 'work': {'title': 'ONE PIECE FILM Z'}, 'status': {'kind': 'watched'}}, {'user': {}, 'action': 'create_status', 'created_at': '2024-12-26T15:36:52Z', 'work': {'title': 'ストライク・ザ・ブラッド'}, 'status': {'kind': 'watched'}}, {'user': {}, 'action': 'create_status', 'created_at': '2024-12-21T00:13:41Z', 'work': {'title': '究極進化した